In [ ]:
#Goal is to find the worst performing products
import numpy as np

#import dataset
import pandas as pd
df = pd.read_excel("Clean_Data.xlsx", sheet_name="Sheet1")

#remove whitespace from column names for easy access
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('$', 'usd') for c in df.columns]
print(df.columns)

In [ ]:
#Create a dictionary that maps:
#Warehouse (Division) -> Unique products sold at division

warehouses = df.legacy_division_cd.unique()
warehouse_to_prod = {}

for w in warehouses:
    warehouse_to_prod[w] = []

for w, p, sc, sales, costs, qsold, nOH in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.sales_channel, 
                                         df.Sales_6_mos, df.cogs_6mos, df.qty_6mos, df.net_OH):
    if p not in warehouse_to_prod[w] and sc == "Warehouse" and sales >= 0 and costs >= 0 and qsold >= 0 and nOH > 0:
        warehouse_to_prod[w].append(p)

# #Print warehouse to number of individual different products stored            
# for w in warehouses:
#     print(w, len(warehouse_to_prod[w]))

In [ ]:
#Create a dictionary of warehouse & product -> total sales in a 6month period
wp_to_sales = {}
#Create a dictionary of warehouse & product -> total costs in a 6month period
wp_to_costs = {}
#Create a dictionary of warehouse & product -> total number of picks in a 6month period
wp_to_picks = {}
#Create a dictionary of warehouse & product -> total quantity sold in a 6month period
wp_to_quantity = {}
#Create a dictionary of warehouse & product -> number of customers sold to at that warehouse
wp_to_ncustomers = {}
#Create a dictionary of warehouse & product -> pallet quantity
wp_to_pallet = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = []
        wp_to_costs[w,p] = []
        wp_to_picks[w,p] = []
        wp_to_quantity[w,p] = []
        wp_to_ncustomers[w,p] = []

for w, p, s, c, pk, q, cust in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, 
                                   df.Sales_6_mos, df.cogs_6mos, df.picks_6mos, df.qty_6mos, df.LEGACY_CUSTOMER_CD):
    if p in warehouse_to_prod[w]:
        wp_to_sales[w,p].append(s)
        wp_to_costs[w,p].append(c)
        wp_to_picks[w,p].append(pk)
        wp_to_quantity[w,p].append(q)
        wp_to_ncustomers[w,p].append(cust)

for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = sum(wp_to_sales[w,p])
        wp_to_costs[w,p] = sum(wp_to_costs[w,p])
        wp_to_picks[w,p] = sum(wp_to_picks[w,p])
        wp_to_quantity[w,p] = sum(wp_to_quantity[w,p])
        wp_to_ncustomers[w,p] = len(wp_to_ncustomers[w,p])


#Create a dictionary of warehouse & product -> coreflag "Y" or "N"
wp_to_coreflag = {}
for w,p,cf in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.core_item_flag):
    if p in warehouse_to_prod[w]:
        if cf == "Y":
            wp_to_coreflag[w,p] = 1
        if cf == "N":
            wp_to_coreflag[w,p] = 0


#Create a dictionary of warehouse & product -> margin%
wp_to_margin = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        s = wp_to_sales[w,p]
        c = wp_to_costs[w,p]
#         ##DATA HAS TO BE CLEANED SO THAT COSTS THAT ARE EQUAL TO 0 DO NOT EXIST
#         if s == 0 or c==0:
#             wp_to_margin[w,p] = 0
#         else:
        wp_to_margin[w,p] = 100*((s-c)/s)


In [ ]:
#Create a Dictionary of a warehouse, product cd pair with its net OH in units, net OH $ as values, item poi days & DIOH
#Example:
#Warehouse 19 houses product 10012415 and it has 4 units in on hand inventory, $81.2 of on hand inventory,
#302.430555555556 of average poi in days & 38.5263664820611 of DIOH
#The dictionary will look like this: wp_to_oh[19, 10012415] = [4.0, 81.2, 302.430555555556, 38.5263664820611]
wp_to_stats = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_stats[w,p] = [0,0,0,0]

for w, p, oh_usd, oh, poi, DIOH in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.net_OH_usd.values, df.net_OH.values,
                           df.item_poi_days.values, df.DIOH.values):
    if p in warehouse_to_prod[w]:
        if wp_to_stats[w,p] == [0,0,0,0]:
            wp_to_stats[w,p] = [oh, oh_usd, poi, DIOH]

In [ ]:
#Create a Dictionary of warehouse & product to turn and earn
wp_to_TE = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        #netOH in quantity
        net_oh = wp_to_stats[w,p][0]
        #cost of goods sold
        cogs = wp_to_costs[w,p]
        #quantity sold
        q = wp_to_quantity[w,p]
        #margin percentage
        mp = wp_to_margin[w,p]
        
        #turn = (2*cogs)/(2*net_oh + q)
        turn = cogs/net_oh
        wp_to_TE[w,p] = turn * mp    

In [ ]:
#Create a Dictionary of warehouse & product -> total profit
wp_to_profit = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_profit[w,p] = wp_to_sales[w,p] - wp_to_costs[w,p]

In [ ]:
for w in warehouses:
    print(w,'|', len(warehouse_to_prod[w]))

In [ ]:
#Vector Model of Data

#Which percentage should we mark as core?
cutoff = 0.20
#Which warehouse do you want to see results for?
warehouse_to_view = 81

#Dictionary of warehouse & Prod -> vector model
wp_to_vector = {}
#Create a dictionary of warehouse & prod to our flag
wp_to_flag = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_vector[w,p] = [wp_to_TE[w,p], wp_to_profit[w,p], wp_to_ncustomers[w,p]] # * weights vec
        wp_to_vector[w,p] = np.linalg.norm(wp_to_vector[w,p]) # TODO - redo to weighted vec length


cutoff = 1- cutoff
for w in warehouses:
    prod_to_score = {}
    for p in warehouse_to_prod[w]:
        prod_to_score[p] = wp_to_vector[w,p]
    #sort products inside of a warehouse by their score in ascending order
    prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
    cutoffIdx = int(len(prods_by_score)*cutoff)
    #the bottom cutoff% of the warehouse & product combination sorted by 
    non_core_prods = prods_by_score[:cutoffIdx]
    core_prods = prods_by_score[cutoffIdx:]
    for p in non_core_prods:
        wp_to_flag[w,p] = 0
    for p in core_prods:
        wp_to_flag[w,p] = 1

#Result statistics
#core products
core = []
#non core products
non_core = []

for p in warehouse_to_prod[warehouse_to_view]:
    if wp_to_flag[warehouse_to_view,p] == 0:
        non_core.append(p)
    else:
        core.append(p)

#Average profit
core_avg_profit = []
non_core_avg_profit = []
#Average turn and Earn
core_avg_TE = []
non_core_avg_TE = []
#Average number of customers
core_avg_ncust = []
non_core_avg_ncust = []

for p in core:
    core_avg_profit.append(wp_to_profit[warehouse_to_view,p])
    core_avg_TE.append(wp_to_TE[warehouse_to_view,p])
    core_avg_ncust.append(wp_to_ncustomers[warehouse_to_view,p])
core_avg_profit = np.average(core_avg_profit)
core_avg_TE = np.average(core_avg_TE)
core_avg_ncust = np.average(core_avg_ncust)

for p in non_core:
    non_core_avg_profit.append(wp_to_profit[warehouse_to_view,p])
    non_core_avg_TE.append(wp_to_TE[warehouse_to_view,p])
    non_core_avg_ncust.append(wp_to_ncustomers[warehouse_to_view,p])
non_core_avg_profit = np.average(non_core_avg_profit)
non_core_avg_TE = np.average(non_core_avg_TE)
non_core_avg_ncust = np.average(non_core_avg_ncust)


#Average profit
avg_profit = []
#Average TE
avg_TE = []
#Average number of customers
avg_ncust = []
for p in warehouse_to_prod[warehouse_to_view]:
    avg_profit.append(wp_to_profit[warehouse_to_view,p])
    avg_TE.append(wp_to_TE[warehouse_to_view, p])
    avg_ncust.append(wp_to_ncustomers[warehouse_to_view, p])
avg_profit = np.average(avg_profit)
avg_TE = np.average(avg_TE)
avg_ncust = np.average(avg_ncust)
    

print('For warehouse', warehouse_to_view)
print()
print("Number of core items", len(core))
print("Core Items Average Profit", core_avg_profit)
print("Core Items Average TE", core_avg_TE)
print('Core Items Average number of customers', core_avg_ncust)
print()
print("Number of non core items", len(non_core))
print("Non Core Items Average Profit", non_core_avg_profit)
print('Non Core Items Average TE', non_core_avg_TE)
print("Non Core Items Average number of customers", non_core_avg_ncust)
print()
print('All Items in warehouse average profit', avg_profit)
print('All Items in warehouse average TE', avg_TE)
print('All Items in warehouse average number of customers', avg_ncust)


In [ ]:
#Print our flag and their flag
num1 = 0
num2 = 0
num3 = 0
num4 = 0
for p in warehouse_to_prod[warehouse_to_view]:
    our_flag = wp_to_flag[warehouse_to_view,p]
    veritiv_flag = wp_to_coreflag[warehouse_to_view,p]
    if our_flag == 1 and veritiv_flag == 0:
        num1 += 1
    if our_flag == 0 and veritiv_flag == 1:
        num2 += 1
    if our_flag == veritiv_flag:
        num3 += 1
    if our_flag == 1:
        num4 += 1
          
print("For Warehouse", warehouse_to_view)
print("Number of times we marked product as core and they marked it as non core", num1)
print("Number of times we marked as non core and they marked as core", num2)
print("Number of times we had the same conclusion", num3)
print("Total number of products in warehouse", len(warehouse_to_prod[warehouse_to_view]))

print("Number of products we marked as core", num4)
print("Number of products we marked as noncore", len(warehouse_to_prod[warehouse_to_view]) - num4)

In [ ]:
w = warehouse_to_view
print("Warehouse:", w)
print()
for p in warehouse_to_prod[w]:
    if wp_to_flag[w,p] == 0 and wp_to_coreflag[w,p]:
        print("Product:", p)
        print("Our Core:",wp_to_flag[w,p], 
              "Veritiv Core:", wp_to_coreflag[w,p], "Sales:", wp_to_sales[w,p], 
              "Costs:", wp_to_costs[w,p], "Margin", wp_to_margin[w,p],"%",
             "Net OH:", wp_to_stats[w,p][0], "Quantity Sold:", wp_to_quantity[w,p])
        print()
